(1) Calculate daily return (return = log(today close/previous close)) 

- Download Candles Data

In [1]:
import yfinance as yf

candles_df = yf.Ticker('THRM.V').history(interval='1d',
                                         start='2018-1-1',
                                         end='2021-1-1')                                    

- Compute the log returns

In [2]:
import numpy as np

candles_df['return'] = candles_df['Close'] / candles_df['Close'].shift(1)
candles_df['log_return'] = np.log(candles_df['return'])
# drop na, i.e., drop first day since no previous close based on the dataset
log_return = candles_df['log_return'].dropna() 

log_return

Date
2018-01-03    0.000000
2018-01-04   -0.068993
2018-01-05   -0.074108
2018-01-08    0.207639
2018-01-09    0.223144
                ...   
2020-12-23   -0.023530
2020-12-24    0.068993
2020-12-29    0.000000
2020-12-30    0.000000
2020-12-31   -0.045462
Name: log_return, Length: 752, dtype: float64

(2)  Conduct the hypothesis testing to check if the distribution of daily return is normal

- $H_0$: Log returns follows normal distribution vs $H_a$: Log returns doesn't follow normal distribution


- Method1: Shapiro-Wilk Test:

In [3]:
from scipy import stats

shapiro_test_res = stats.shapiro(log_return)
print('The p-value for Shapiro-Wilk test is {}'.format(
    shapiro_test_res.pvalue))

The p-value for Shapiro-Wilk test is 4.011051164214806e-30


Since the p-value < 0.05, we reject $H_0$, which implies that the log returns do not follow the normal distribution.

- Method2: KS-Test (Compare ECDF and Taregeted CDF, i.e., normal distribution)

In [4]:
est_mean = log_return.mean()
est_sd = (log_return.var() * len(log_return) / (len(log_return) - 1))**.5

ks_test_res = stats.kstest(
    log_return, lambda x: stats.norm.cdf(x, loc=est_mean, scale=est_sd))
print('The p-value for K-S test is {}'.format(ks_test_res.pvalue))

The p-value for K-S test is 7.179832123503775e-60


Since the p-value < 0.05, we reject $H_0$, which implies that the log returns do not follow the normal distribution.